# Autogen with GEMINI

In [1]:
!pip install git+https://github.com/microsoft/autogen.git@v0.2.25

  Cloning https://github.com/microsoft/autogen.git (to revision v0.2.25) to /tmp/pip-req-build-xsoo9rwz
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/autogen.git /tmp/pip-req-build-xsoo9rwz
  Running command git checkout -q 4ab8a884870f4aeafe3587c56169bb094061af5b
  Encountered 1 file(s) that should have been pointers, but weren't:
        website/static/img/gallery/autotx.png
  Resolved https://github.com/microsoft/autogen.git to commit 4ab8a884870f4aeafe3587c56169bb094061af5b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import os
api_key = os.environ.get("GEMINI_API_KEY")

In [4]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.code_utils import  infer_lang
from google.colab import userdata

# Retrieve your API key securely stored as a secret
api_key= userdata.get('GEMINI_API_KEY')

if not api_key:
  raise ValueError(f"KEY: '{api_key}' is invalid")

# MAX_USER_REPLIES=5
# INPUT_START_MESSAGE = """
# write an article about how pewdiepie became the most subscribed youtuber.
# }
# """

# Define your model configuration
config_list_gemini = [
    {
        "model": "gemini-1.5-flash-latest",
        "api_key": api_key,
        "api_type": "google"
    }
]

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=config_list_gemini[0],
    system_message="""Engineer. You follow an approved plan. You write python code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

scientist = autogen.AssistantAgent(
    name="Analyst",
    llm_config=config_list_gemini[0],
    system_message="""Analyst. You follow an approved plan. You are able to draw conclusion from codes and graphs. You don't write code.""",
)
#########################################
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and an Analystt who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a Analyst.
""",
    llm_config=config_list_gemini[0],
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },
)


groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=30
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list_gemini[0])


In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
Should a businessman invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks using FB prophet library. If stock prices are increasing for next 5 days, invest in Apple stocks.
""",
)

Admin (to chat_manager):


Should a businessman invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks using FB prophet library. If stock prices are increasing for next 5 days, invest in Apple stocks.


--------------------------------------------------------------------------------
Planner (to chat_manager):

## Plan: Apple Stock Investment Recommendation

**Objective:** Determine if a businessman should invest in Apple stocks based on a 5-day time series forecast using the FB Prophet library.

**Roles:**

* **Analyst:** Responsible for data gathering, cleaning, and preparing data for analysis. 
* **Engineer:** Responsible for building and running the Prophet model, interpreting results, and generating the investment recommendation.

**Steps:**

1. **Data Gathering (Analyst):**
    * Obtain historical Apple stock price data for the past 3 years (or longer if available). This can be done through financial data APIs, websites like Yahoo Finance, or databa